# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier


In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Вывод:** В файле присутствуют 3214 строк с 4 количественными признаками о звонках, их суммарной длительности в минутах, потраченных мегабайтах интернета и используемом тарифе. Пустых строк нет, предварительная обработка была сделана ранее.

## Разбейте данные на выборки

Разобъем исходную выборку на обучающую, валидационную и тестовую в соотношении 3:1:1 соответственно.

In [ ]:
df_train, df_test = train_test_split(df,train_size=0.6, random_state=12345)
df_test, df_valid = train_test_split(df_test, train_size=0.5,random_state=12345)

In [ ]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


**Вывод:** Полученные выборки удовлетворяют необходимым соотношениям и размерам.

## Исследуйте модели

Для исследование были выбраны модели случайного леса, логистическая регрессия и дерево решений.

### Случайный лес

In [ ]:
best_model_random_forest = None
best_result = 0
for est in range(1, 101):
    for d in range(1,16):
        for ml in range(1,11):
            model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=d, min_samples_leaf=ml) # обучите модель с заданным количеством деревьев
            model.fit(features_train,target_train) # обучите модель на тренировочной выборке
            result = model.score(features_valid,target_valid) # посчитайте качество модели на валидационной выборке
            if result > best_result:
                best_model_random_forest = model
                best_result = result #  сохраните наилучшее значение метрики accuracy на валидационных данных
                best_est = est
                best_md = d
                best_ml = ml

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Количество деревьев наилучшей модели на валидационной выборке:", best_est)
print("Максимальная глубина наилучшей модели на валидационной выборке:", best_md)
print("Минимальное количество объектов в листьях наилучшей модели на валидационной выборке:", best_md)

### Логистическая регрессия

In [ ]:
best_model_log_regression = None
best_result = 0
#solver = ['newton-cg','lbfgs','liblinear','sag','saga']
penalty = ['l2','none']
C = [0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]

for p in penalty:
    for c in C:
        model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=10000, penalty=p, C=c) # обучите модель с заданным количеством деревьев
        model.fit(features_train,target_train) # обучите модель на тренировочной выборке
        result = model.score(features_valid,target_valid) # посчитайте качество модели на валидационной выборке
        if result > best_result:
            best_model_log_regression = model
            best_result = result #  сохраните наилучшее значение метрики accuracy на валидационных данных
            best_penalty = p
            best_c = c

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Penalty наилучшей модели на валидационной выборке:", best_penalty)
print("C наилучшей модели на валидационной выборке:", best_c)

### Дерево решений

In [ ]:
best_model_tree_classifier = None
best_result = 0

for depth in range(1, 101):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth) 
    model.fit(features_train,target_train)
    result = model.score(features_valid,target_valid) # посчитайте качество модели на валидационной выборке
    if result > best_result:
        best_model_tree_classifier = model
        best_result = result#  сохраните наилучшее значение метрики accuracy на валидационных данных
        best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Глубина наилучшей модели на валидационной выборке:", best_depth)

**Вывод:** Были рассмотрены три модели: случайноо леса, логистическая регресия и дерево решений. Для каждой модел изменялись ее гиперпараметры для получения наилучшой модели. В результате наилучшая доля правильных ответов достигнута при использовании случайного леса (0.82). В дальнейшем будем использовать именно эту модель.

## Проверьте модель на тестовой выборке

Проверим лучшую модель случайного леса на тестовой выборке.

In [ ]:
predictions_test = best_model_random_forest.predict(features_test)

print("Тестовая выборка:", accuracy_score(target_test, predictions_test))

**Вывод:** выбранная модель показала долю правильных ответов на тестовой выорке равную 0.79, что соответствует заданию.

## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность воспользуемся простейшей моделью, которая всегда предсказывает наиболее часто встречающийся класс. Так мы получим контрольные данные для сравнительной оценки построенной вами более сложной модели.

In [ ]:
model_dummy_classifiery = DummyClassifier(random_state=12345) 
model_dummy_classifiery.fit(features_train,target_train)
predictions_test_dummy = model_dummy_classifiery.predict(features_test)
print("Тестовая выборка:", accuracy_score(target_test, predictions_test_dummy))

**Вывод:** наша выбранная сложная модель показала большую точность (0.79), чем простейшая (0.71)

## Вывод

В данном проекте было необходимо построить модель для задачи классификации, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В качестве признаков были взяты данные о поведении клиентов, которые уже перешли на эти тарифы. В файле присутствуют 3214 строк с 4 количественными признаками о звонках, их суммарной длительности в минутах, потраченных мегабайтах интернета и используемом тарифе. Пропусков нет, предварительная обработка была сделана ранее.Исходная выборка была разбита на обучающую, валидационную и тестовую в соотношении 3:1:1 соответственно.

Были рассмотрены три модели: случайноо леса, логистическая регресия и дерево решений. Для каждой модел изменялись ее гиперпараметры для получения наилучшой модели. 
1. Лучшая модель *случайного леса* дотигла доли правильных ответов 0.82 при гиперпараметрах: количество деревьев = 6, максимальная глубина = 11, а минимальное количество объектов в листьях = 11. 
2. Лучшая модель *логистической регресии* дотигла доли правильных ответов 0.74 при гиперпараметрах: штраф = l2, C = 0.2.
3. Лучшая модель *дерево решений* дотигла доли правильных ответов 0.8 при глубине = 7.
В результате наилучшая доля правильных ответов достигнута при использовании случайного леса (0.82). 

Было проведено тестирование выбранной модели на тестовой выборке, где она показала долю правильных ответов  равную 0.79, что соответствует изначальным требованиеям.

Для проверки модели на адекватность воспользовались простейшей моделью, которая всегда предсказывает наиболее часто встречающийся класс. Так мы получили контрольные данные для сравнительной оценки построенной вами более сложной модели. В результате сравнения наша выбранная сложная модель показала большую точность (0.79), чем простейшая (0.71)

Таким образом, была построена модель для задачи классификации определения предпочтительного тарифа среди пользователй с нужной долей правльных ответов.